In [2]:
!git clone https://github.com/AmauryTavares/ReconhecimentoCAPTCHA.git

fatal: destination path 'ReconhecimentoCAPTCHA' already exists and is not an empty directory.


## Reconhecimento de CAPTCHA


### Pacotes necessários para o projeto

In [0]:
import cv2 as cv
import numpy as np
import os
import keras
import csv
from zipfile import ZipFile
from PIL import Image
from IPython.display import Image

from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization

### Funções para pré-processamento da imagem

In [0]:
def getSegments(sorted_contours, binary_openning):
    rois = []
    img_segmented = cv.cvtColor(binary_openning, cv.COLOR_GRAY2BGR)
    for i, ctr in enumerate(sorted_contours):
        # obtem pontos inicial, altura e largura do retangulo
        x, y, w, h = cv.boundingRect(ctr)

        # obtem parte segmentada
        roi = binary_openning[y:y + h, x:x + w]
        rois.append(roi)

        # mostra parte segmentada
        cv.rectangle(img_segmented, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return rois, img_segmented
  
def averageROI(rois):
    average = 0;
    for roi in rois:
        average += roi.shape[1]

    average = (average/len(rois))
    return average
  
#remove outliers de segmentos de ruidos menores
def removeMinimumOutliersROI(rois):
    newRois = []
    for roi in rois:
        if roi.shape[0] * roi.shape[1] > 100:
            newRois.append(roi)

    return newRois


#segmenta os rois maiores
def separateSegment(rois):
    newRois = []
    average = np.int32(averageROI(rois) * 1.4)

    for roi in rois:
        sizeBigger = np.int32(roi.shape[1]/average)

        if sizeBigger == 0:
            newRois.append(roi)
        else:
            sizeBigger += 1
            width = np.int32(roi.shape[1]/sizeBigger)
            height = roi.shape[0]
            y = 0
            x = 0
            for i in range(sizeBigger):
                newRois.append(roi[y:height, x:width * (i + 1)])
                x += width

    return newRois

In [0]:
def preProcessing(path):
    # Mascara para usada na operacao morfologica
    kernel = np.ones((3, 3), np.int8)

    #carreg imagem original
    src = cv.imread(path, cv.IMREAD_GRAYSCALE)

    # converte imagem para tom de cinza
    src_gray = cv.imread(path, cv.IMREAD_GRAYSCALE)

    # binarizacao da imagem
    ret, binary = cv.threshold(src_gray, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    # operacao morfologica de fechamento
    binary_openning = cv.morphologyEx(binary, cv.MORPH_CLOSE, kernel)

    # imagem negativa
    binary_negative = cv.bitwise_not(binary_openning)

    # obtem os contornos do captcha
    _, contours, _ = cv.findContours(binary_negative, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # ordena os contornos no eixo x
    sorted_contours = sorted(contours, key=lambda ctr: cv.boundingRect(ctr)[0])

    rois, img_segmented = getSegments(sorted_contours, binary_openning)
    if len(rois) > 1:
        rois = removeMinimumOutliersROI(rois)
        rois = separateSegment(rois)

    # modifica o tamanho para 28x28
    for i, roi in enumerate(rois):
      rois[i] = cv.resize(roi, (28,28), interpolation=cv.INTER_AREA)
        
    return rois, src

### Dividir base de dados em conjuntos de treinamento, teste e validação.

In [0]:
# Carregar base de dados
#display(Image('2a4f.png'))
def loadDatabase():
  files = []
  filenames = []
  root = './ReconhecimentoCAPTCHA/images/'
  # carregar nomes dos captchas
  with open('./ReconhecimentoCAPTCHA/filenames.csv', 'r') as readFile:
    reader = csv.reader(readFile)
    filenames = list(reader)[0]

  for filename in filenames:
    files.append(root + filename)
   
  # divide base de dados
  training_data, validation_data, test_data = files[:35000], files[35000:42500], files[42500:]
  training_label, validation_label, test_label = filenames[:35000], filenames[35000:42500], filenames[42500:]
    
  #print(len(training_data), 'exemplos de treino')
  #print(len(validation_data), 'exemplos de validação')
  #print(len(test_data), 'exemplos de teste')
    
  return (training_data, validation_data, test_data, training_label, validation_label, test_label)

### Funções para manipulação da base de dados

In [0]:
# Faz o mapeamento de caracter para one hot
def mapAlphanumeticToOnehot(caracter):
  alphanumeric = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
                  'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
                  'u', 'v', 'x', 'y', 'z', 'w']
  
  for x, _ in enumerate(alphanumeric):
    if caracter == alphanumeric[x]:
      alphanumeric[x] = 1
    else:
      alphanumeric[x] = 0
  
  return alphanumeric

def convertCaracterToOnehot(label):
  newLabel = np.zeros((len(label), 36))
  for i, x in enumerate(label):
    newLabel[i, :] = mapAlphanumeticToOnehot(x)
  return newLabel

def expandDatabase(x_train, y_train, flatten=False):
  if flatten:
    rois_x_train = np.zeros((4*(len(x_train)), 784))
  else:
    rois_x_train = np.zeros((4*(len(x_train)), 28, 28, 1))
        
  rois_y_train = []
  i = 0
  
  for x, y in zip(x_train, y_train):
    rois, _ = preProcessing(x)

    for roi in rois:
      roi = roi.astype('float32')/255
      
      if flatten:
        rois_x_train[i, :,] = roi.reshape((784))
      else:
        rois_x_train[i, :, :, :] = roi.reshape((28,28,1))
      
      i += 1

    filename = y.split('.')[0]
    
    for x in filename:
      rois_y_train.append(x)
      
  print('Pre-processamento concluído!')
    
  return rois_x_train, rois_y_train

### Modelo LeNet

In [0]:
def build_LeNet(width, height, depth, classes):
  model = Sequential()
  inputShape = (height, width, depth)

  model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape, activation="relu"))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

  model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

  model.add(Flatten())
  model.add(Dense(500,  activation="relu"))

  model.add(Dense(classes))
  model.add(Activation("softmax"))
  
  model.summary()
  
  return model

### Modelo MiniVGG

In [0]:
def build_MiniVGG(width, height, depth, classes):
  model = Sequential()
  inputShape = (height, width, depth)
  
  model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape, activation="relu"))
  model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  

  model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
  model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  model.add(Dense(512, activation="relu"))
  model.add(Dropout(0.5))
  
  model.add(Dense(classes))
  model.add(Activation("softmax"))
  
  model.summary()
  
  return model

### Modelo VGG16

In [0]:
def build_VGG16(width, height, depth, classes):
  model = Sequential()
  inputShape = (height, width, depth)
  
  model.add(Conv2D(64, (3, 3), padding="same", activation="relu", input_shape=inputShape))
  model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2)))
  
  model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
  model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2)))
  
  model.add(Conv2D(256, (3, 3), padding="same", activation="relu"))
  model.add(Conv2D(256, (3, 3), padding="same", activation="relu"))
  model.add(Conv2D(256, (3, 3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2)))
  
  model.add(Conv2D(512, (3, 3), padding="same", activation="relu"))
  model.add(Conv2D(512, (3, 3), padding="same", activation="relu"))
  model.add(Conv2D(512, (3, 3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2)))
  
  model.add(Conv2D(256, (3, 3), padding="same", activation="relu"))
  model.add(Conv2D(256, (3, 3), padding="same", activation="relu"))
  model.add(Conv2D(256, (3, 3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(1,1)))
  
  model.add(Flatten())
  model.add(Dense(4096, activation="relu"))
  model.add(Dense(4096, activation="relu"))
  model.add(Dense(classes, activation="softmax"))
  
  model.summary()
  
  return model

### Modelo AlexNet

In [0]:
def build_AlexNet(width, height, depth, classes):
  model = Sequential()
  inputShape = (height, width, depth)
  
  model.add(Conv2D(64, (3, 3), padding="same", activation="relu", input_shape=inputShape))
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
  
  model.add(Conv2D(128, (3, 3), padding="same", activation="relu", strides=(1,1)))
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
  
  model.add(Conv2D(256, (3, 3), padding="same", activation="relu", strides=(1,1)))
  model.add(Conv2D(256, (3, 3), padding="same", activation="relu", strides=(1,1)))
  model.add(Conv2D(256, (3, 3), padding="same", activation="relu", strides=(1,1)))
  model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
  
  model.add(Flatten())
  model.add(Dense(4096, activation="relu"))
  model.add(Dropout(0.4))
  
  model.add(Dense(4096, activation="relu"))
  model.add(Dropout(0.4))
  
  model.add(Dense(1000, activation="relu"))
  model.add(Dropout(0.4))
  
  model.add(Dense(classes, activation="softmax"))
  
  model.summary()
  
  return model

### Modelo MLP (4 Camadas relu)

In [0]:
def build_MLP4relu(inputShape, classes):
  model = Sequential()
  
  model.add(Dense(512, activation='relu', input_shape=(inputShape,)))
  model.add(Dropout(0.4))
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(classes, activation='softmax'))
  
  model.summary()
  
  return model

### Modelo MLP (4 Camadas Tanh)

In [0]:
def build_MLP4tanh(inputShape, classes):
  model = Sequential()
  
  model.add(Dense(512, activation='tanh', input_shape=(inputShape,)))
  model.add(Dropout(0.4))
  model.add(Dense(512, activation='tanh'))
  model.add(Dropout(0.4))
  model.add(Dense(classes, activation='softmax'))
  
  model.summary()
  
  return model

### Modelo MLP (7 Camadas relu)

In [0]:
def build_MLP7relu(inputShape, classes):
  model = Sequential()
  
  model.add(Dense(512, activation='relu', input_shape=(inputShape,)))
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(classes, activation='softmax'))
  
  model.summary()
  
  return model

### Modelo MLP (7 Camadas tanh)

In [0]:
def build_MLP7tanh(inputShape, classes):
  model = Sequential()
  
  model.add(Dense(512, activation='tanh', input_shape=(inputShape,)))
  model.add(Dense(256, activation='tanh'))
  model.add(Dense(128, activation='tanh'))
  model.add(Dense(64, activation='tanh'))
  model.add(Dropout(0.4))
  model.add(Dense(64, activation='tanh'))
  model.add(Dropout(0.4))
  model.add(Dense(classes, activation='softmax'))
  
  model.summary()
  
  return model

### Treino do modelo

In [0]:
def trainModel(model, flatten=False):
  (x_train, x_valid, _, y_train, y_valid, _) = loadDatabase()

  x_train, y_train = expandDatabase(x_train, y_train, flatten)
  x_valid, y_valid = expandDatabase(x_valid, y_valid, flatten)
  y_train = convertCaracterToOnehot(y_train)
  y_valid = convertCaracterToOnehot(y_valid)

  hist = model.fit(x_train, y_train, batch_size=128, epochs=30,
            validation_data=(x_valid, y_valid), verbose=2, shuffle=True)
    
  return hist

### Calculando a acurácia do modelo

In [0]:
def accuracyModel(model, flatten=False):
  (_, _, x_test, _, _, y_test) = loadDatabase()
  
  x_test, y_test = expandDatabase(x_test, y_test, flatten)
  y_test = convertCaracterToOnehot(y_test)

  score = model.evaluate(x_test, y_test, verbose=0)
  print('\nTaxa de acerto:', score[1])

### Calculando acurácia de CAPTCHA do modelo

In [0]:
def accuracyCAPTCHA(model, flatten=False):
  (_, _, x_test, _, _, y_test) = loadDatabase()
  length_test = len(x_test)
  x_test, y_test = expandDatabase(x_test, y_test, flatten)
  y_test = convertCaracterToOnehot(y_test)
  
  predictions = model.predict(x_test, batch_size=4)
  result = predictions.argmax(axis=1)
  result_test = y_test.argmax(axis=1)

  totalHit = 0
  onlyOneMiss = 0

  for i in range(length_test):
    hit = 0
    for x in range(i*4, (i+1)*4):
      if (result[x] == result_test[x]):
        hit += 1

    if hit == 4:
      totalHit += 1
      onlyOneMiss += 1
    elif hit == 3:
      onlyOneMiss += 1

  print('Taxa de acerto (4 caracteres):', totalHit / length_test)
  print('Taxa de acerto (3 caracteres):', onlyOneMiss / length_test)

### Informações sobre o treinamento do modelo

In [0]:
# # evaluate the network
# print("[INFO] evaluating network...")

# # get predictions on the test set
# predictions = model.predict(x_test, batch_size=128)

# # define text labels (source: https://www.cs.toronto.edu/~kriz/cifar.html)
# cifar10_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


# print(classification_report(y_test.argmax(axis=1),
# predictions.argmax(axis=1),
# target_names=cifar10_labels))

# plot the training loss and accuracy
def infoTrain(hist):
  plt.style.use("ggplot")
  plt.figure()
  plt.plot(np.arange(0, 30), hist.history["loss"], label="train_loss" )
  plt.plot(np.arange(0, 30), hist.history["val_loss"], label="val_loss")
  plt.plot(np.arange(0, 30), hist.history["acc" ], label="train_acc" )
  plt.plot(np.arange(0, 30), hist.history["val_acc"], label="val_acc")
  plt.title("Training Loss and Accuracy")
  plt.xlabel("Epoch #")
  plt.ylabel("Loss/Accuracy")
  plt.legend()
  plt.show()

### Configuração e executando o modelo Lenet

In [0]:
opt = SGD(lr=0.01)
model = build_LeNet(width=28, height=28, depth=1, classes=36)
model.compile(loss='categorical_crossentropy', optimizer=opt, 
                  metrics=['accuracy'])

hist = trainModel(model)
accuracyModel(model)
accuracyCAPTCHA(model)
infoTrain(hist)

### Configuração e executando o modelo MiniVGG

In [0]:
opt = SGD(lr=0.01)
model = build_MiniVGG(width=28, height=28, depth=1, classes=36)
model.compile(loss='categorical_crossentropy', optimizer=opt, 
                  metrics=['accuracy'])

hist = trainModel(model)
accuracyModel(model)
accuracyCAPTCHA(model)
infoTrain(hist)

### Configuração e executando modelo VGG16

In [0]:
#opt = SGD(lr=0.01)
#model = build_VGG16(width=28, height=28, depth=1, classes=36)
#model.compile(loss='categorical_crossentropy', optimizer=opt, 
#                  metrics=['accuracy'])

#hist = trainModel(model)
#accuracyModel(model)
#accuracyCAPTCHA(model)
#infoTrain(hist)

### Configuração e executando modelo AlexNet

In [0]:
opt = SGD(lr=0.01)
model = build_AlexNet(width=28, height=28, depth=1, classes=36)
model.compile(loss='categorical_crossentropy', optimizer=opt, 
                  metrics=['accuracy'])

hist = trainModel(model)
accuracyModel(model)
accuracyCAPTCHA(model)
infoTrain(hist)

### Configuração e executando MLP 4 Camadas relu

In [0]:
opt = SGD(lr=0.01)
model = build_MLP4relu(inputShape=784, classes=36)
model.compile(loss='categorical_crossentropy', optimizer=opt, 
                  metrics=['accuracy'])

hist = trainModel(model, flatten=True)
accuracyModel(model, flatten=True)
accuracyCAPTCHA(model, flatten=True)
infoTrain(hist)

### Configuração e executando MLP 4 Camadas tanh

In [0]:
opt = SGD(lr=0.01)
model = build_MLP4tanh(inputShape=784, classes=36)
model.compile(loss='categorical_crossentropy', optimizer=opt, 
                  metrics=['accuracy'])

hist = trainModel(model, flatten=True)
accuracyModel(model, flatten=True)
accuracyCAPTCHA(model, flatten=True)
infoTrain(hist)

### Configuração e executando MLP 7 Camadas relu

In [0]:
opt = SGD(lr=0.01)
model = build_MLP7relu(inputShape=784, classes=36)
model.compile(loss='categorical_crossentropy', optimizer=opt, 
                  metrics=['accuracy'])

hist = trainModel(model, flatten=True)
accuracyModel(model, flatten=True)
accuracyCAPTCHA(model, flatten=True)
infoTrain(hist)

### Configuração e executando MLP 7 Camadas tanh

In [0]:
opt = SGD(lr=0.01)
model = build_MLP7tanh(inputShape=784, classes=36)
model.compile(loss='categorical_crossentropy', optimizer=opt, 
                  metrics=['accuracy'])

hist = trainModel(model, flatten=True)
accuracyModel(model, flatten=True)
accuracyCAPTCHA(model, flatten=True)
infoTrain(hist)